### Read Review Translations
Note: if the reviews haven't been translated, run the notebook 'Translate_Reviews' to generate the input table

In [28]:
# Read input data from Delta table
df = spark.sql( \
"""
SELECT
  text, text_translated
FROM
  translated_reviews
""")

display(df)

StatementMeta(, 5813e152-280b-4f7b-80ea-0d91477f9a4e, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, e66cbe06-0f89-40a5-b790-6857ae8839ea)

### Fetch Azure AI Service Key from Key Vault

In [29]:
# Get Azure AI Services Keys
from trident_token_library_wrapper \
import PyTridentTokenLibrary as tl

key_vault_name = 'designmind-fabric-ai'
key_name = "AZURE-AI-SERVICES-KEY" 

# Get access token to key vault for current session ID
access_token = mssparkutils.credentials.getToken("keyvault")

# Get secret value from Key Vault using the access token
ai_services_key = tl.get_secret_with_token( \
  f"https://{key_vault_name}.vault.azure.net/", \
  key_name, \
  access_token)
ai_services_region = "eastus"

StatementMeta(, 5813e152-280b-4f7b-80ea-0d91477f9a4e, 31, Finished, Available)

In [30]:
print(ai_services_key)


StatementMeta(, 5813e152-280b-4f7b-80ea-0d91477f9a4e, 32, Finished, Available)

[REDACTED]


### Apply Azure AI Sentiment Analysis Model to text_tanslated

In [33]:
# Append Sentiment to DataFrame
from synapse.ml.services import *

sentiment = (
    TextSentiment()
    .setTextCol("text")
    .setOutputCol("sentiment_result")
    .setLocation(ai_services_region)
    .setSubscriptionKey(ai_services_key)
    .setErrorCol("error")
)

df = sentiment.transform(df)
display(df)

StatementMeta(, 5813e152-280b-4f7b-80ea-0d91477f9a4e, 35, Finished, Available)

SynapseWidget(Synapse.DataFrame, 76e4e748-03ef-42da-b050-f5bc16a3ebfc)

In [34]:
df = df.select(
    df.columns[0:2] + ["sentiment_result.document.sentiment"]
)
display(df)

StatementMeta(, 5813e152-280b-4f7b-80ea-0d91477f9a4e, 36, Finished, Available)

SynapseWidget(Synapse.DataFrame, a291b2a0-2577-416c-a070-75abdac5cdd6)

### Save to Delta table in Data Lake

In [35]:
df.write.mode("overwrite").format("delta").saveAsTable("review_sentiment")

StatementMeta(, 5813e152-280b-4f7b-80ea-0d91477f9a4e, 37, Finished, Available)

### Review Table

In [36]:
%%sql
SELECT * FROM review_sentiment;

StatementMeta(, 5813e152-280b-4f7b-80ea-0d91477f9a4e, 38, Finished, Available)

<Spark SQL result set with 5 rows and 3 fields>